In [4]:
import firebase_admin
from firebase_admin import db, credentials, firestore

In [8]:
cred = credentials.Certificate("credentials.json")
firebase_admin.initialize_app(cred)

In [9]:
db = firestore.client()

In [32]:


# Load CSV file into DataFrame
file_path = "data/freelancer_tax_deductions.csv"  # Adjust the path
df = pd.read_csv(file_path)

# Ensure "transaction_id" is a string and handle missing values
df["transaction_id"] = df["transaction_id"].astype(str).fillna("unknown")

# Firestore collection name
collection_name = "tax_deductions"

# Upload each row to Firestore
for _, row in df.iterrows():
    doc_ref = db.collection(collection_name).document(row["transaction_id"])
    doc_ref.set(row.to_dict())

print("Data successfully uploaded to Firebase Firestore!")


RetryError: Timeout of 60.0s exceeded, last exception: 503 Getting metadata from plugin failed with error: ('invalid_grant: Invalid JWT: Token must be a short-lived token (60 minutes) and in a reasonable timeframe. Check your iat and exp values in the JWT claim.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT: Token must be a short-lived token (60 minutes) and in a reasonable timeframe. Check your iat and exp values in the JWT claim.'})

In [11]:

def save_transaction(user_id, category, amount, payment_method, merchant):
    transaction_ref = db.collection("users").document(user_id).collection("transactions").document()

    transaction_data = {
        "category": category,
        "amount": amount,
        "payment_method": payment_method,
        "merchant": merchant,
        "timestamp": firestore.SERVER_TIMESTAMP,
    }

    transaction_ref.set(transaction_data)
    print(f"Transaction saved for {user_id}: {transaction_data}")


In [13]:
def get_user_transactions(user_id):
    transactions_ref = db.collection("users").document(user_id).collection("transactions")
    docs = transactions_ref.stream()

    user_transactions = []
    for doc in docs:
        transaction_data = doc.to_dict()
        transaction_data["id"] = doc.id  # Include transaction ID if needed
        user_transactions.append(transaction_data)

    return user_transactions

In [ ]:
def update_deduction_limit(user_id, category, amount_used):
    deduction_ref = db.collection("users").document(user_id).collection("deductions").document(category)
    deduction_doc = deduction_ref.get()

    if deduction_doc.exists:
        data = deduction_doc.to_dict()
        new_used = data["used_limit"] + amount_used
        new_remaining = max(0, data["max_limit"] - new_used)

        deduction_ref.update({
            "used_limit": new_used,
            "remaining_limit": new_remaining,
        })
    else:
        deduction_ref.set({
            "category": category,
            "max_limit": 1000,  # Default limit (customizable)
            "used_limit": amount_used,
            "remaining_limit": 1000 - amount_used,
        })

    print(f"Updated deduction for {user_id} - {category}: Used {amount_used}")


In [12]:
from datetime import datetime

def save_tax_report(user_id, tax_savings):
    report_ref = db.collection("users").document(user_id).collection("tax_reports").document()

    report_data = {
        "year": datetime.utcnow().year,
        "total_savings": tax_savings,
        "generated_at": datetime.utcnow(),
        "report_link": f"https://docs.google.com/document/d/{report_ref.id}/edit",
    }

    report_ref.set(report_data)
    print(f"Tax report saved for {user_id}: {report_data}")

    return report_data["report_link"]
